In [ ]:
# Go to Menu > Runtime > Change runtime.
# Change hardware acceleration to GPU.

# This will take ~20min to complete
!git clone https://github.com/deep-privacy/sa-toolkit
!cd sa-toolkit && ./install.sh

Cloning into 'sa-toolkit'...
remote: Enumerating objects: 3935, done.
remote: Counting objects: 100% (3935/3935), done.
remote: Compressing objects: 100% (1830/1830), done.
remote: Total 3935 (delta 2168), reused 3560 (delta 1793), pack-reused 0
Receiving objects: 100% (3935/3935), 85.55 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (2168/2168), done.
 == Google colab / Kaggle detected, running py37 | Warning: Performing /usr/local OVERWRITE! ==
Using local $CUDAROOT: /usr/local/cuda
Cuda version: 111
 - Downloading a pre-compiled version of kaldi
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0    791      0 --:--:-- --:--:-- --:--:--   791
100   133  100   133    0     0    259      0 --:--:-- --:--:-- --:--:--   259
100   651  100   651    0     0    849      0 --:--:-- --:--:-- --:--:--   849
100  927M  100  927M    0     0  4764k      0  0:

In [ ]:
%%bash
cd sa-toolkit/pkwrap/egs/librispeech/v1
wget -q https://huggingface.co/Champion/SA-models/resolve/main/e2e_tdnnf_wav2vec_fairseq_hibitrate.zip
yes no | unzip ./e2e_tdnnf_wav2vec_fairseq_hibitrate.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/e2e_tdnnf.zip
yes no | unzip ./e2e_tdnnf.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/decode.zip
yes no | unzip decode.zip

rm *.zip
cd -

cd sa-toolkit/pkwrap/egs/LJSpeech
wget -q https://huggingface.co/Champion/SA-models/resolve/main/hifigan_w2w2.zip
yes no | unzip hifigan_w2w2.zip

wget -q https://huggingface.co/Champion/SA-models/resolve/main/hifigan_tdnnf.zip
yes no | unzip hifigan_tdnnf.zip

cd -

wget -q https://huggingface.co/Champion/SA-models/resolve/main/LibriTTS_f0_stats.zip
yes no | unzip LibriTTS_f0_stats.zip
rm LibriTTS_f0_stats.zip

Archive:  ./e2e_tdnnf_wav2vec_fairseq_hibitrate.zip
  inflating: exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/den.fst  
  inflating: exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/final.mdl  
  inflating: exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/0.trans_mdl  
  inflating: exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/final.pt  
  inflating: exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/normalization.fst  
 extracting: exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/num_pdfs  
  inflating: exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/tree  
Archive:  ./e2e_tdnnf.zip
  inflating: exp/chain/e2e_tdnnf/den.fst  
  inflating: exp/chain/e2e_tdnnf/final.mdl  
  inflating: exp/chain/e2e_tdnnf/0.trans_mdl  
  inflating: exp/chain/e2e_tdnnf/final.pt  
  inflating: exp/chain/e2e_tdnnf/normalization.fst  
 extracting: exp/chain/e2e_tdnnf/num_pdfs  
  inflating: exp/chain/e2e_tdnnf/tree  
Archive:  decode.zip
  inflating: exp/chain/e2e_biphone_tree/graph_tgsmall/HCLG.fst  
  inflating: exp/chain

replace exp/chain/e2e_tdnnf/0.trans_mdl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import sys, os
from pathlib import Path
PYTHON_PACKAGE="/usr/local/lib/python3.7/site-packages/" # ATM: Colab is currently running on python3.7
# import python package installed with miniconda
sys.path.insert(0,PYTHON_PACKAGE)
for egg in list(Path(PYTHON_PACKAGE).glob(f"**/*egg-link")):
  sys.path.append(open(egg).read().splitlines()[0])

# TODO: Set LD_LIBRARY_PATH accordingly to  sa-toolkit/env.sh and kaldi path.sh
# os.environ['LD_LIBRARY_PATH'] = open("sa-toolkit/env.sh").read().splitlines()[0].split("LD_LIBRARY_PATH=")[1].split(";")[0]
# os.environ['LD_LIBRARY_PATH'] += f":{os.getcwd()}/sa-toolkit/kaldi/src/lib"
# os.environ['KALDI_ROOT'] = f":{os.getcwd()}/sa-toolkit/kaldi/"

In [ ]:
import torch
import torchaudio
import pkwrap
import pkwrap.infer_helper as demo
import json
from IPython.display import Audio, HTML, Javascript

<IPython.core.display.Javascript object>

In [ ]:

forward_asr_wav2vec2, pk_model = demo.init_asr_model(
    model="local/chain/e2e/tuning/tdnnf_wav2vec_fairseq_hibitrate.py",
    exp_path="exp/chain/e2e_tdnnf_wav2vec_fairseq_hibitrate/",
)
# forward_asr_wav2vec2, pk_model = demo.init_asr_model(
#     model="local/chain/e2e/tuning/tdnnf.py",
#     exp_path="exp/chain/e2e_tdnnf/",
# )

Loading 'exp/chain/e2e_tdnnf/final.pt'


CRITICAL:root:pkwrap: -- Warning using bypass on the TDNNF_LD layer!
CRITICAL:root:pkwrap: -- Failed to import kaldi you better not be in training mode (no backward possible) --


In [ ]:
forward_synt, synt_model = demo.init_synt_hifigan_w2v2(
    model=f"local/tuning/hifi_gan_wav2vec2.py",
    exp_path=f"exp/hifigan_w2w2",
    asr_bn_model=pk_model,
    model_weight="g_00050000",
    json_stats_file="stats.json"
)
# forward_synt, synt_model = demo.init_synt_hifigan_w2v2(
#     model=f"local/tuning/hifi_gan_tdnnf.py",
#     exp_path=f"exp/hifigan_tdnnf",
#     asr_bn_model=pk_model,
#     model_weight="g_00111000",
#     json_stats_file="stats.json"
# )

Loading 'exp/hifigan_tdnnf/g_00111000'


In [ ]:
from google.colab.output import eval_js
import base64
import tempfile

# Test with your voice
display(Javascript(demo.record_audio_colab))
output = eval_js('recordAudio({})')
tmpf = tempfile.NamedTemporaryFile(suffix='.wav')
tmpf.write(base64.b64decode(output))
print('Recording saved to:', tmpf.name, "\nNow running ffmpeg..")
tmpfwav = tempfile.NamedTemporaryFile(suffix='.wav').name
get_ipython().getoutput(f"ffmpeg -i {tmpf.name} -ac 1 -f wav {tmpfwav} -y -hide_banner -loglevel panic")
tmpf.close()
waveform, sr = torchaudio.load(tmpfwav)
waveform = torchaudio.transforms.Resample(orig_freq=sr,
                                                   new_freq=16000)(waveform)
display(Audio(data=waveform.cpu(), rate=16000))

<IPython.core.display.Javascript object>

Recording saved to: /tmp/tmpjom6g985.wav 
Now running ffmpeg..


In [ ]:
# Or some file on the disk
file = "own.mp3"
tmpfwav = tempfile.NamedTemporaryFile(suffix='.wav').name
get_ipython().getoutput(f"ffmpeg -i {file} -ac 1 -f wav {tmpfwav} -y -hide_banner -loglevel panic")
waveform, sr = torchaudio.load(tmpfwav)
waveform = torchaudio.transforms.Resample(orig_freq=sr,
                                                   new_freq=16000)(waveform)
display(Audio(data=waveform.cpu(), rate=16000))

In [ ]:
# Or use libri speaker
!wget -q https://huggingface.co/proxy-datasets-preview/assets/librispeech_asr/--/clean/validation/0/audio/audio.mp3 -O audio.mp3
waveform, sr = torchaudio.load("audio.mp3")

In [ ]:
# ASR result from the ASR model
display(Audio(data=waveform, rate=16000))
out, net = forward_asr_wav2vec2(waveform.to(demo.device).clone())
text = demo.kaldi_asr_decode(out)
text_gt = "HE WAS IN A FEVERED STATE OF MIND OWING TO THE BLIGHT HIS WIFE'S ACTION THREATENED TO CAST UPON HIS ENTIRE FUTURE "
_,_, repport = demo.computeWER(text, text_gt)
print(text, "\n")
display(HTML(repport))

AH THESE DON BENITO PRESENTATION OR UNION OF TWO VIVA VOCE BITTER OPPOSITION 



In [ ]:
#@title Select the target for voice conversion
f0_stats = json.loads(open("./stats.json",'r').readline())
spkids = list(f0_stats.keys())
target = '7190' #@param ['7190', '4481', '8324', '7402', '5808', '8123', '1447', '3240', '6081', '8088', '125', '831', '4788', '6385', '8838', '1069', '7226']
# Good ones: 5808 6081 7109

# And use the ASR accoustic model to extract bottleneck feature
# Then generate a new voice
def _norm(f0, f0_stats, filename):
  return pkwrap.hifigan.f0.m_std_norm(f0, f0_stats[target], filename)
pkwrap.hifigan.f0.set_norm_func(_norm)

f0 = pkwrap.hifigan.f0.get_f0(
    waveform.cpu().clone(), f0_stats=f0_stats
).to(demo.device)

audio = forward_synt(audio=waveform.to(demo.device).clone(), f0=f0, target=[target])
display(Audio(data=audio, rate=16000))